### Conll to note format for sanskrit

In [4]:
# # Rename the files to gold and silver
# import argparse
# import os
# path = './data/train_20k/'
# count = 0
# for file in os.listdir(path):
#         count= count + 1
#         dst ="Silver_" + str(count) + ".csv"
#         src =file
#         os.rename(path+src,path+dst) 

In [2]:
import argparse
import os
path = './sanskrit_treebank/'
languages_for_low_resource = ['san']
languages = sorted(list(set(languages_for_low_resource)))
#
splits = ['train_20k']
lng_to_files = dict((language, {}) for language in languages)
for language, d in lng_to_files.items():
    for split in splits:
        d[split] = []
    lng_to_files[language] = d
sub_folders = os.listdir(path)
for sub_folder in sub_folders:
    if sub_folder != 'train_20k':
        continue
    folder = os.path.join(path, sub_folder)
    files = sorted(os.listdir(folder))
    for file in files:
        full_path = os.path.join(folder, file)
        lng_to_files[language][sub_folder].append(full_path)
tag = set()
for language, split_dict in lng_to_files.items():
    posi =set()
    for split, files in split_dict.items():
        sentences = []
        num_sentences = 0
        for file in files:
            num_sentences += 1
            flag = 0
            with open(file, 'r') as file:
                for line in file:
                    new_line = []
                    line = line.strip()
#                     print(line)
                    if flag == 0:
                        flag = 1
                        continue
                    tokens = line.split(',')
                    id = str(int(tokens[0]) + 1)
                    word = tokens[1]
                    ############################
                    # To add rule features id of rule is given
                    lemma = tokens[2]
                    pos = tokens[3]
                    tag.add(pos)
                    if tokens[6]:
                        arc_tag = tokens[5]
                        head = str(int(tokens[6]) + 1)
                    else:
                        arc_tag = 'root'
                        head = '0'
                    new_line = [id, word, pos, lemma, head, arc_tag]
                    posi.add(pos)
#                     print(new_line)
                    sentences.append(new_line)
            sentences.append([])   
        
        print('Language: %s Split: %s Num. Sentences: %s ' % (language, split, num_sentences))
        if not os.path.exists('data'):
            os.makedirs('data')
        write_data_path = 'sanskrit_treebank/ud_pos_ner_dp_' + split + '_' + language
        print('creating %s' % write_data_path)
        with open(write_data_path, 'w') as f:
            for line in sentences:
                f.write('\t'.join(line) + '\n')

Language: san Split: train_20k Num. Sentences: 20297 
creating sanskrit_treebank/ud_pos_ner_dp_train_20k_san


### Build 1-hot for POS embedding:

In [19]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
# data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
data = list(tag)
values = array(data)
# print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
print('Dimension is',len(onehot_encoded[0]))
# # invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

Dimension is 475


/home/jivnesh/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [25]:
# onehot_encoded[0]
count =0 
with open("pos_embedding_1_hot.475", "w") as f:   
    f.write(str(len(tag))+' '+str(475)+'\n')
    for word in tag:
        word=word+'@'
        f.write(word) 
        for v in onehot_encoded[count]:
            f.write(' '+str(v))
        f.write('\n')
        count = count + 1
print(count)

475


In [11]:
total

22500

In [5]:
s = 0
for a,b in splits:
    s = s+ a + b
s

27375

In [6]:
s-total

4875

In [10]:
# gold = 2495
# silver = 20297
gold = 2500
silver = 20000
total = gold+silver
batchsize = 250
sums = 0
deduction = 5
count = 0
inter = 1
splits = list()
while(sums<total):
    while(count<5):
        sums+=batchsize
        gold -=batchsize
        print(inter,batchsize,0)
        splits.append([batchsize,0])
#         print('Gold',gold)
#         print('Silver',silver)
        inter+=1
        count +=1
    if gold-deduction and deduction<batchsize :
        print(inter,batchsize-deduction,deduction)
        splits.append([batchsize-deduction,deduction])
        inter+=1
        gold -= (batchsize-deduction)
        silver -= deduction
#         print('Gold',gold)
#         print('Silver',silver)
        sums+=batchsize
        deduction += 5
    else:
#         print('Gold',gold)
# #         print(inter,gold,batchsize- gold)
#         inter+=1
#         silver -= batchsize- gold
# #         print('Gold',gold)
# #         print('Silver',silver)
        gold = 0
        
        sums+=batchsize
    while gold == 0 and silver- batchsize> 0:
        print(inter,0,batchsize)
        splits.append([0,batchsize])
        inter+=1
        silver -= batchsize
        sums+=batchsize
#         print('Gold',gold)
#         print('Silver',silver)
    if silver- batchsize< 0:
        print(inter,0,silver)
        splits.append([0,silver])
        break

1 250 0
2 250 0
3 250 0
4 250 0
5 250 0
6 245 5
7 240 10
8 235 15
9 230 20
10 225 25
11 220 30
12 215 35
13 210 40
14 205 45
15 200 50
16 195 55
17 190 60
18 185 65
19 180 70
20 175 75
21 170 80
22 165 85
23 160 90
24 155 95
25 150 100
26 145 105
27 140 110
28 135 115
29 130 120
30 125 125
31 120 130
32 115 135
33 110 140
34 105 145
35 100 150
36 95 155
37 90 160
38 85 165
39 80 170
40 75 175
41 70 180
42 65 185
43 60 190
44 55 195
45 50 200
46 45 205
47 40 210
48 35 215
49 30 220
50 25 225
51 20 230
52 15 235
53 10 240
54 5 245
55 0 250
56 0 250
57 0 250
58 0 250
59 0 250
60 0 250
61 0 250
62 0 250
63 0 250
64 0 250
65 0 250
66 0 250
67 0 250
68 0 250
69 0 250
70 0 250
71 0 250
72 0 250
73 0 250
74 0 250
75 0 250
76 0 250
77 0 250
78 0 250
79 0 250
80 0 250
81 0 250
82 0 250
83 0 250
84 0 250
85 0 250
86 0 250
87 0 250
88 0 250
89 0 250
90 0 250
91 0 250
92 0 250
93 0 250
94 0 250
95 0 250
96 0 250
97 0 250
98 0 250
99 0 250
100 0 250
101 0 250
102 0 250
103 0 250
104 0 250
105 0 250
